# LSTM-250 Model Export as SQLite3 Database

## Goals

- Loads Dataset and Model from Numpy notebook.
- Model and dataset exported as sqlite3 database for implementation in C.

**NOTE:** The dataset exported by the training notebook may have incorrect predicted index due to several iterations of model training and not updating the dataset. We'll re-run the predictions here and update the predicted index in the dataset.

In [ ]:
import torch
import os
import numpy as np
import IPython.display as ipd
from tqdm.auto import tqdm

print('Using PyTorch version:', torch.__version__)

# Load Dataset and the Model Parameters

In [ ]:
np_model_path = './session/numpy-model-dataset.pt'
loaded_dict = torch.load(np_model_path)
print(loaded_dict.keys())

# Extract the items from the dictionary
Accuracy = loaded_dict['accuracy']
Correct_count = loaded_dict['correct_count']
Hparam = loaded_dict['Hparam']
Index_to_label = loaded_dict['index_to_label']
Weights_np = loaded_dict['Weights_np']
DataItems = loaded_dict['DataItems']

# Delete redundant names
del np_model_path, loaded_dict

# Export Numpy Model as sqlite3 DB

In [ ]:
# Delete the cache and import sqlite3 utilities
!rm -rf __pycache__/
from utilsqlite3 import *

In [ ]:
# Create the database file
DB_path = './saved/trained-lstm250.s3db'
createDB(DB_path, overwrite=True)
!ls -ltrh ./saved

## Write the Header Table

In [ ]:
# Creates the header table
def createHeaderTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = '''CREATE TABLE IF NOT EXISTS Header (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        key TEXT,
                        value NUMERIC,
                        description TEXT
                    )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createHeaderTable(DB_path)
table_names = getTableNames(DB_path)
print(table_names)
del table_names

In [ ]:
# Inserts a record into the Header table (will be called by insertRecordList() utility function)
def insertHeaderRecord(cursor, record):
    key, value, description = record  # this serves as a soft check for the record
    # Insert the record into the table
    cursor.execute('''INSERT INTO Header (key, value, description)
                      VALUES (?, ?, ?)''', (key, value, description))


# Call the function to insert a record
insertRecordList(DB_path, insertHeaderRecord, [('example_key', 'example_value', 'example_description')])
getRecords(DB_path, 'Header')

In [ ]:
# Make valid table names (cannot have . in their name)
# Table name: lstm/fc_weight/bias_xx_lx
print(Weights_np.keys())
Weight_tables = {}
for key, param in Weights_np.items():
    tname = key.replace('.', '_')          # replace any remaining '.' with '_'
    tkey  = key+'.table'                   # key for the Header table
    Weight_tables[tkey] = (tname, param)   # save as (tname, param) to be used later
    print(f'{tkey:23}', ':', tname)

# Delete redundant names
del key, param, tname

In [ ]:
# Define the records as a list
Hparam_table = 'Hparam_T'

header_records = [
    ('name', 'LSTM-250', ''),
    ('architecture', '123-LSTM:250-LSTM:250-LSTM:250-FC:39', 'It is an LSTM with 3 layers with 250 units with a fully-connected layer at the output with 39 classes. Trained on TIMIT dataset for phoneme classification.'),
    ('accuracy', Accuracy, 'Accuracy% of the trained model on the test dataset.'),
    ('correct_count', Correct_count, 'Number of correct predictions by the trained model on the test dataset.'),
    ('Hparam.table',   Hparam_table, 'This is the name of the table that contains different parameters of the model.'),
]
for k,v in Weight_tables.items():
    header_records.append( (k, v[0], 'Name of the table containing the parameter ' + k.replace('.table','')) )


# Insert the header records
deleteRows(DB_path, 'Header')  # delete previous records
insertRecordList(DB_path, insertHeaderRecord, header_records)
records = getRecords(DB_path, 'Header')
for r in records: print(r[:-1])   # print all but description field
    
# Delete names
del records, k, v

## Write the Hparam Table

In [ ]:
# Creates the Hparam table
def createHparamTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = f'''CREATE TABLE IF NOT EXISTS {Hparam_table} (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        key TEXT,
                        value NUMERIC,
                        description TEXT
                    )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createHparamTable(DB_path)
table_names = getTableNames(DB_path)
print(table_names)

# Delete neames
del table_names

In [ ]:
# Inserts a record into the Hparam table
def insertHparamRecord(cursor, record):
    key, value, description = record  # this serves as a soft check for the record
    # Insert the record into the table
    cursor.execute(f'''INSERT INTO {Hparam_table} (key, value, description)
                      VALUES (?, ?, ?)''', (key, value, description))


# Call the function to insert a record
insertRecordList(DB_path, insertHparamRecord, [('example_key', 123, 'example_description')])
getRecords(DB_path, Hparam_table)

In [ ]:
# Insert the Hparam records
print('Hparam from pytorch:', Hparam)

hparam_records = [
    ('input_size', Hparam['input_size'], 'Input size of the LSTM'),
    ('hidden_size', Hparam['hidden_size'], 'Size of hidden states the LSTM layers'),
    ('num_layers', Hparam['num_layers'], 'Number of LSTM layers'),
    ('num_classes', Hparam['num_classes'], 'Output size of the Fully-Connected output layer'),
]
for k, v in Weights_np.items():
    hparam_records.append( (k+'.shape', str(v.shape), 'Dimensions of the parameter '+k) )
    

deleteRows(DB_path, Hparam_table)
insertRecordList(DB_path, insertHparamRecord, hparam_records)
getRecords(DB_path, Hparam_table)

## Write Weights and Biases

In [ ]:
# Saves a numpy 2D array as a table in the database.
# Columns: ID, row_no, col_0, col_1, ..., col_n
def createMatrixTable(db_path, table_name, nparray, overwrite=False):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists and overwrite requested
    table_exist = existTable(db_path, table_name)
    if overwrite and table_exist:
        print(f'WARN: Overwriting table {table_name}')
        dropTable(db_path, table_name)
        
    # Create the table
    rows, cols = nparray.shape
    column_names = "row_no, " + ", ".join([f"col_{i}" for i in range(cols)])  # Generate the column names string
    cursor.execute(f"CREATE TABLE {table_name} ({column_names})")

    # Insert the array rows into the table
    for i in range(rows):
        vals = f'{i}, ' + ', '.join(map(str, nparray[i]))
        cursor.execute(f"INSERT INTO {table_name} VALUES ({vals})")

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# test createMatrixTable()
createMatrixTable(DB_path, 'test', Weights_np['fc.weight'], overwrite=True)
col_names = getColNames(DB_path, 'test')
records = getRecords(DB_path, 'test')
print('col_names:', col_names[:5], '...', col_names[-5:])
print('records[i]:', records[2][:5], '...')

# Delete names
del col_names, records

In [ ]:
# Save the weights
for k,v in Weight_tables.items():
    tname, params = v    # extract the table name and parameters
    # Convert 1D arrays to 2D array if necessary
    if len(params.shape) == 1: 
        params = np.expand_dims(params, axis=0)
    # create the table
    createMatrixTable(DB_path, tname, params, overwrite=True)
    print(f'INFO: Created table {tname}')


print('')
print(getTableNames(DB_path))

del k, v, tname, params

In [ ]:
# Drop extra tables
keep_tables = {'sqlite_sequence', 'Header', 'Hparam_T', 
               'lstm_weight_ih_l0', 'lstm_weight_hh_l0', 'lstm_bias_ih_l0', 'lstm_bias_hh_l0', 
               'lstm_weight_ih_l1', 'lstm_weight_hh_l1', 'lstm_bias_ih_l1', 'lstm_bias_hh_l1', 
               'lstm_weight_ih_l2', 'lstm_weight_hh_l2', 'lstm_bias_ih_l2', 'lstm_bias_hh_l2', 
               'fc_weight', 'fc_bias'}

all_tables = getTableNames(DB_path)
cnt = 0
for name in all_tables:
    if name not in keep_tables:
        dropTable(DB_path, name)
        print(f'WARN: Dropped table {name}')
        cnt += 1
print(f'INFO: {cnt} tables dropped')

all_tables = getTableNames(DB_path)
ipd.display(all_tables)

# Delete names
del all_tables, cnt, name, keep_tables

## Import the Saved Model and Validate

In [ ]:
# Check the meta tables
rec_list = getRecords(DB_path, 'Header')
print('Header:')
for r in rec_list: print(r[1:-1])

print('')
rec_list = getRecords(DB_path, 'Hparam_T')
print('Hparam:')
for r in rec_list: print(r[1:-1])

In [ ]:
# Returns a table saved using createMatrixTable as a list of tuples
def readMatrixTable(db_path, table_name):
    # read the records
    rec_list = getRecords(db_path, table_name)
    # build the matrix
    rec_list.sort()         # sort by row_no (first column)
    matrix = []
    for rec in rec_list:
        matrix.append(rec[1:])  # stripe off the row_no columns
    return matrix


# test this functions
mat1 = np.array(readMatrixTable(DB_path, 'fc_weight'))
mat1.shape

In [ ]:
# Returns the weights and biases as a dictionary
def readModelParam(db_path, table_names):
    model_params = {}
    for name in table_names:
        # read the matrix as a list of tuples
        mat = readMatrixTable(db_path, name)
        # Check if it is a matrix or a vector
        if len(mat)==1: is_vector = True
        else: is_vector = False
        # convert to numpy array
        if is_vector: mat = np.array(mat[0])    # make a 1D array for vectors
        else: mat = np.array(mat)
        # save it for returning
        model_params[name] = mat
    return model_params
        

# test this function
param_tables = [v[0] for v in Weight_tables.values()]   # get the parameter table names

model_params = readModelParam(DB_path, param_tables)
for k, v in model_params.items():
    print(f'{k}:', v.shape, v.dtype)
    

# Delete names
del k, v, mat1

In [ ]:
# Compare with original weights
org_params = {v[0]:v[1] for v in Weight_tables.values()}    # make weight table for comparison


def compare_model_params(model_params, org_params, tolerance):
    for k in model_params:
        print('\nComparing:', k)
        db_val = model_params[k]
        org_val = org_params[k]
        dmin = np.min(org_val)
        dmax = np.max(org_val)
        print('min:', dmin, '  max:', dmax)
        diff_val = np.max(np.abs(db_val - org_val))   # get the maximum difference
        print('diff:', diff_val)
        assert np.allclose(db_val, org_val, rtol=tolerance)  # use numpy built-in check


# compare with tolerance
tolerance = 1e-6
compare_model_params(model_params, org_params, tolerance)

In [ ]:
del tolerance, tkey, rec_list, r, org_params, param_tables, model_params
del hparam_records, header_records

# Export the Dataset as sqlite3 DB

In [ ]:
# Create the database file
DB_ds_path = './saved/timit_test_data.s3db'
createDB(DB_ds_path, overwrite=True)
!ls -ltr saved/

## Save the Header Table

In [ ]:
# Create the table and check 
createHeaderTable(DB_ds_path)
table_names = getTableNames(DB_ds_path)
print(table_names)

In [ ]:
# Define the records as a list
Label_table = 'Labels_T'
Dataitem_table = 'DataItems_T'
FeatureSeq_table = 'FeatureSequences_T'

item = DataItems[0]
print('item[-1] shape:', item[-1].shape)    # last field in the item is the feature-sequence
Feature_len = item[-1].shape[1]

header_records = [
    ('name', 'TIMIT-Test', 'Feature-Sequences extracted from the test dataset of TIMIT.'),
    ('feature_length', Feature_len, 'The length of the feature. These features can be directly fed to the MLP12 model'),
    ('accuracy', Accuracy, 'Accuracy of the LSTM250 model used to generate the "predicted_index" values.'),
    
    ('labels.table',  Label_table, 'Index to label mapping. The model predicts an index, which can be converted to the label using this table'),
    ('dataset.table', Dataitem_table, 'This table serves as the (label, feature) list. The actual features are stored in a separate table.'), 
    ('dataitem.schema', '', 'There are 3 label-related fields in the dataset.table: "label" is the ground-truth, "label_index" is the index into the index-to-label mapping, "predicted_index" is the index predicted by the trained LSTM250 model'),
    ('feature_sequence.table', FeatureSeq_table, 'Contains the actual feature-sequences for the model.'),
]

# Insert the header records
deleteRows(DB_ds_path, 'Header')  # delete previous records
insertRecordList(DB_ds_path, insertHeaderRecord, header_records)
records = getRecords(DB_ds_path, 'Header')
for r in records: print(r[:-1])   # print all but description field

## Save Index-to-label mapping table

In [ ]:
# Creates the labels table
def createLableTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = f'''CREATE TABLE IF NOT EXISTS {Label_table} (
                    label_index INTEGER PRIMARY KEY,
                    label TEXT
                )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createLableTable(DB_ds_path)
table_names = getTableNames(DB_ds_path)
print(table_names)

In [ ]:
# Inserts a record into the labels table
def insertLabelRecord(cursor, record):
    label_index, label = record  # this serves as a soft check for the record format
    # Insert the record into the table
    query = f"INSERT INTO {Label_table} (label_index, label) VALUES (?, ?)"
    cursor.execute(query, (label_index, label))


# Call the function to insert a record
deleteRows(DB_ds_path, Label_table)
insertRecordList(DB_ds_path, insertLabelRecord, [(-1, 'test')])
getRecords(DB_ds_path, Label_table)

In [ ]:
# Build the label records
label_records = [(label_index, str(label)) for label_index, label in Index_to_label.items()]
print('label_records:', label_records)

# Store them in the table
deleteRows(DB_ds_path, Label_table)
insertRecordList(DB_ds_path, insertLabelRecord, label_records)
rec_list = getRecords(DB_ds_path, Label_table)
print('rec_list:', rec_list)

In [ ]:
del table_names, records, rec_list, r, label_records, item

## Separate Data-items and Feature Sequences

In [ ]:
# Split the dataitems for DataItem table and Features table
dataitem_records = []
featureSeq_records = []

item = DataItems[0]
print('item[-1].type:', type(item[-1]))

for item_index, item in enumerate(DataItems):
    label, label_index, pred_index, seq_len, feat_seq = item   # parse the item
    seq_id = item_index      # use the index in the dataset as the feature-sequence ID
    item_rec = [label, label_index, pred_index, seq_len, seq_id]
    feat_rec = (seq_id, feat_seq)    # feature-record: (feature-id, feature-sequence)
    dataitem_records.append(item_rec)
    featureSeq_records.append(feat_rec)

# check the records
print('')
print('dataitem_records:', len(dataitem_records), len(dataitem_records[0]))
print('featureSeq_records:', len(featureSeq_records), featureSeq_records[0][1].shape)

# Check the feature-sequences
check_index = 100
item = DataItems[check_index]
npmatch = (featureSeq_records[check_index][1] == item[-1])
assert npmatch.all()

In [ ]:
del seq_id, seq_len, pred_index, npmatch, label_index, label, item_rec
del item_index, item, header_records, feat_rec, feat_seq, check_index

## Save Data Items

In [ ]:
# Creates the dataset table to save the data-items
def createDataTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Define the table name and column names
    table_name = Dataitem_table
    columns = ["id INTEGER PRIMARY KEY AUTOINCREMENT",
               "label TEXT",
               "label_index INTEGER",
               "predicted_index INTEGER",
               "sequence_len INTEGER",
               "sequence_id INTEGER"]

    # Create the table
    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)})"
    cursor.execute(query)
    conn.commit()
    conn.close()


# Create the table
createDataTable(DB_ds_path)
table_names = getTableNames(DB_ds_path)
print(table_names)

In [ ]:
# Inserts a record into the Dataset table
def insertDataRecord(cursor, record):
    label, label_index, predicted_index, seq_len, seq_id = record  # this serves as a soft check for the record
    # Insert the record into the table
    query = f"INSERT INTO {Dataitem_table} (label, label_index, predicted_index, sequence_len, sequence_id) VALUES (?, ?, ?, ?, ?)"
    cursor.execute(query, (label, label_index, predicted_index, seq_len, seq_id))


# Call the function to insert a record
insertRecordList(DB_ds_path, insertDataRecord, [("Item 1", 1, 2, 3, 4)])
getRecords(DB_ds_path, Dataitem_table)

In [ ]:
# insert all dataset records
deleteRows(DB_ds_path, Dataitem_table)   # delete old records
insertRecordList(DB_ds_path, insertDataRecord, dataitem_records)
rec_list = getRecords(DB_ds_path, Dataitem_table)
print('rec_list:', len(rec_list), len(rec_list[0]))
print('rec_list[0]', rec_list[3])

In [ ]:
del table_names, rec_list

## Save Feature Sequences

In [ ]:
# Creates the feature-sequence table to save the feature_sequences
# Columns: sequence-id, row-index, col_0, col_1, ..., col_n
# row-index is the the index of the corresponding row in the sequence.
def createFeatureTable(db_path, table_name, sequence_records, overwrite=False):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists and overwrite requested
    table_exist = existTable(db_path, table_name)
    if overwrite and table_exist:
        print(f'WARN: Overwriting table {table_name}')
        dropTable(db_path, table_name)
        
    # Create the table
    feat_vec = sequence_records[0][1][0]  # get a feature vector
    cols = len(feat_vec)                  # no. of columns to create in the table
    column_names = "sequence_id, row_index, " + ", ".join([f"col_{i}" for i in range(cols)])  # Generate the column names string
    cursor.execute(f"CREATE TABLE {table_name} ({column_names})")

    # Insert the featurs into the table
    for seq_item in tqdm(sequence_records):
        seq_id, feat_seq = seq_item
        for row_id, feat_vec in enumerate(feat_seq):
            vals = f'{seq_id}, {row_id}, ' + ', '.join(map(str, feat_vec))
            cursor.execute(f"INSERT INTO {table_name} VALUES ({vals})")

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table
createFeatureTable(DB_ds_path, FeatureSeq_table, featureSeq_records, overwrite=True)
table_names = getTableNames(DB_ds_path)
print(table_names)

rec_list = getRecords(DB_ds_path, FeatureSeq_table)
print('rec_list:', len(rec_list))

In [ ]:
feat_item = rec_list[0]
print('feat_item:', len(feat_item))
type_count = {}
for d in feat_item: 
    t = type(d)
    if t not in type_count: type_count[t] = 0
    type_count[t] += 1
print(type_count)


# Delete names
del feat_item, type_count, d

In [ ]:
# Drop extra tables ---------------
keep_tables = {'Header', 'sqlite_sequence', 'Labels_T', 'DataItems_T', 'FeatureSequences_T'}

all_tables = getTableNames(DB_ds_path)
cnt = 0
for name in all_tables:
    if name not in keep_tables:
        dropTable(DB_ds_path, name)
        print(f'WARN: Dropped table {name}')
        cnt += 1
print(f'INFO: {cnt} tables dropped')

all_tables = getTableNames(DB_ds_path)
ipd.display(all_tables)

# Delete names
del all_tables, cnt, name

# Import Saved Dataset and Verify

In [ ]:
# Check the meta tables
rec_list = getRecords(DB_ds_path, 'Header')
print('Header:')
for r in rec_list: print(r[1:-1])

In [ ]:
# Compare with original features and dataitems
def compare_dataset(db_items, org_items, tolerance):
    range_iter = tqdm(range(len(db_items)))
    for i in range_iter:
        #print(i)
        db_rec = db_items[i]
        org_rec = org_items[i]
        # Compare the labels and sequence lengths
        db_labels = tuple(db_rec[:4])
        org_labels = tuple(org_rec[:4])
        #print('db_rec:', db_rec)
        #print('org_rec:', org_rec)
        assert db_labels==org_labels, f"Labels mismatch:\ndb_labels: {db_labels}\norg_labels:{org_labels}"
        #if i==5: break
        # Check features
        org_feat = np.array(org_rec[-1])
        db_feat = np.array(db_rec[-1])
        assert np.allclose(org_feat, db_feat, tolerance), "Feature vector mismatch"
    print(f'INFO: Compared {(i+1)} records')


# merge the tables to make similar records as in DS_loaded['dataset']
# build a feat_id: feat_vec map for merging.
feat_records = getRecords(DB_ds_path, FeatureSeq_table)
feat_records.sort()    # sort by (seq-id, row-index)
seq_rec_map = {}
for fitem in feat_records:    # fitem: (seq-id, row-index, col_0, col_1 ...)
    seq_id = fitem[0]
    feat_vec = fitem[2:]
    if seq_id not in seq_rec_map:
        seq_rec_map[seq_id] = []
    seq_rec_map[seq_id].append(feat_vec)
    

# merge feature vectors with dataset items for comparison
data_records = getRecords(DB_ds_path, Dataitem_table)
db_items = []
for drec in data_records:
    seq_id = drec[-1]
    feat_seq = np.array(seq_rec_map[seq_id])
    merged_item = list(drec[1:5]) + [feat_seq]   # remove ID column and concat feature vector
    db_items.append(merged_item)
        
        

In [ ]:
# compare with tolerance
tolerance = 1e-6
compare_dataset(db_items, DataItems, tolerance)

# Concluding Remarks

Now you can use these databases to translate the Numpy model into C implementations. You can also run experiments on fixed-point precisions.